<a href="https://colab.research.google.com/github/tdogubo/data-analysis/blob/main/Tweets_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install -q tweepy emot

In [38]:
import os
import nltk
import tweepy
import pandas as pd
import re
import pickle
from getpass import getpass as gp
from emot.emo_unicode import EMOJI_UNICODE, EMOJI_ALIAS_UNICODE, EMOTICONS_EMO

In [39]:
api_key = gp("Input apikey:: ")
api_key_secret = gp("Input apikey secret:: ")
access_token = gp("Input access token:: ")
access_token_secret = gp("Input access token secret:: ")


Input apikey:: ··········
Input apikey secret:: ··········
Input access token:: ··········
Input access token secret:: ··········


In [40]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

In [64]:
public_tweets = api.user_timeline(screen_name ="@TechCabal", count = 1000, exclude_reply = True, include_rts= True)
public_tweets[:3]
len(public_tweets)

200

In [65]:
tweets=[]
for tweet in public_tweets:
  tweets.append([tweet._json["created_at"],tweet._json["text"]])

print(tweets)

[['Tue Feb 22 11:55:03 +0000 2022', 'TechCabal talks with Bunmi Adeoye, who heads investments in the tech, media, and telecom sector at ACA, about the k… https://t.co/dZluryZvDE'], ['Tue Feb 22 10:22:32 +0000 2022', 'After 9 cities across 8 countries in Africa, 150 businesses and over 2,000 individuals working out of its spaces, w… https://t.co/7bIb7IlL4H'], ['Tue Feb 22 09:28:33 +0000 2022', "RT @TechCabal: In this week's edition of #QuickFire, we interview Amel Saidane, a Tunisian entrepreneur who is a key player in North Africa…"], ['Tue Feb 22 09:11:52 +0000 2022', "Q1 '22 is not over yet and many more startups still have funding announcements on the way."], ['Tue Feb 22 09:11:52 +0000 2022', 'About 76% of the funding comes from the Big 4—Nigeria, South Africa, Kenya and Egypt—but Tunisia dealt huge numbers… https://t.co/X58ZZMQFh8'], ['Tue Feb 22 09:11:51 +0000 2022', 'Just two months into 2022 and African startups have already raised a whopping $1 billion across 130 funding deals

In [66]:
tweets_df = pd.DataFrame(tweets, columns =["created_at", "tweets"])
tweets_df

,created_at,tweets
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i..."
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1..."
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...
...,...,...
195,Tue Feb 08 18:36:09 +0000 2022,"Join @arnergy's CEO, @femiadeyemo on #BFGU thi..."
196,Tue Feb 08 17:53:22 +0000 2022,RT @genesystechhub: @TechCabal Canva designers...
197,Tue Feb 08 16:35:03 +0000 2022,Meta said that while it's winding down Express...
198,Tue Feb 08 16:05:48 +0000 2022,"In April 2021, Spotify launched the Spotify EQ..."


In [67]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)
def convert_emojis(text):
      for emot in EMOJI_UNICODE:
          if EMOJI_UNICODE[emot] in text:
            text= text.replace(EMOJI_UNICODE[emot] , emot.replace(":","").replace("_"," "))
      return text
# Example
tweets_df['tweets_without_emojis']= tweets_df['tweets'].apply(lambda x: convert_emojis(x))
tweets_df
# text1 = "Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒"
# convert_emojis(text1)

,created_at,tweets,tweets_without_emojis
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i..."
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1..."
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...
...,...,...,...
195,Tue Feb 08 18:36:09 +0000 2022,"Join @arnergy's CEO, @femiadeyemo on #BFGU thi...","Join @arnergy's CEO, @femiadeyemo on #BFGU thi..."
196,Tue Feb 08 17:53:22 +0000 2022,RT @genesystechhub: @TechCabal Canva designers...,RT @genesystechhub: @TechCabal Canva designers...
197,Tue Feb 08 16:35:03 +0000 2022,Meta said that while it's winding down Express...,Meta said that while it's winding down Express...
198,Tue Feb 08 16:05:48 +0000 2022,"In April 2021, Spotify launched the Spotify EQ...","In April 2021, Spotify launched the Spotify EQ..."


In [68]:
import string
string.punctuation
#defining the function to remove punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
tweets_df['clean_tweets']= tweets_df['tweets_without_emojis'].apply(lambda x:remove_punctuation(x))
tweets_df.head()

,created_at,tweets,tweets_without_emojis,clean_tweets
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i...",TechCabal talks with Bunmi Adeoye who heads in...
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1...",After 9 cities across 8 countries in Africa 15...
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...,RT TechCabal In this weeks edition of QuickFir...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...,Q1 22 is not over yet and many more startups s...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...,About 76 of the funding comes from the Big 4—N...


In [69]:
tweets_df['tweets_lower']= tweets_df['clean_tweets'].apply(lambda x: x.lower())
tweets_df

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i...",TechCabal talks with Bunmi Adeoye who heads in...,techcabal talks with bunmi adeoye who heads in...
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1...",After 9 cities across 8 countries in Africa 15...,after 9 cities across 8 countries in africa 15...
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...,RT TechCabal In this weeks edition of QuickFir...,rt techcabal in this weeks edition of quickfir...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...,Q1 22 is not over yet and many more startups s...,q1 22 is not over yet and many more startups s...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...,About 76 of the funding comes from the Big 4—N...,about 76 of the funding comes from the big 4—n...
...,...,...,...,...,...
195,Tue Feb 08 18:36:09 +0000 2022,"Join @arnergy's CEO, @femiadeyemo on #BFGU thi...","Join @arnergy's CEO, @femiadeyemo on #BFGU thi...",Join arnergys CEO femiadeyemo on BFGU this Fri...,join arnergys ceo femiadeyemo on bfgu this fri...
196,Tue Feb 08 17:53:22 +0000 2022,RT @genesystechhub: @TechCabal Canva designers...,RT @genesystechhub: @TechCabal Canva designers...,RT genesystechhub TechCabal Canva designers ar...,rt genesystechhub techcabal canva designers ar...
197,Tue Feb 08 16:35:03 +0000 2022,Meta said that while it's winding down Express...,Meta said that while it's winding down Express...,Meta said that while its winding down Express ...,meta said that while its winding down express ...
198,Tue Feb 08 16:05:48 +0000 2022,"In April 2021, Spotify launched the Spotify EQ...","In April 2021, Spotify launched the Spotify EQ...",In April 2021 Spotify launched the Spotify EQU...,in april 2021 spotify launched the spotify equ...


In [70]:
#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#applying function to the column
tweets_df['tweets_tokenied']= tweets_df['tweets_lower'].apply(lambda x: tokenization(x))
tweets_df[:5]
tweets_df.shape

(200, 6)

In [71]:
#Stop words present in the library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [72]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

  #applying the function
tweets_df['tweets_no_stopwords']= tweets_df['tweets_tokenied'].apply(lambda x:remove_stopwords(x))
tweets_df[:5]

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i...",TechCabal talks with Bunmi Adeoye who heads in...,techcabal talks with bunmi adeoye who heads in...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1...",After 9 cities across 8 countries in Africa 15...,after 9 cities across 8 countries in africa 15...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...,RT TechCabal In this weeks edition of QuickFir...,rt techcabal in this weeks edition of quickfir...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...,Q1 22 is not over yet and many more startups s...,q1 22 is not over yet and many more startups s...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...,About 76 of the funding comes from the Big 4—N...,about 76 of the funding comes from the big 4—n...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...


In [73]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text

tweets_df['tweets_stemmed']=tweets_df['tweets_no_stopwords'].apply(lambda x: stemming(x))
tweets_df[:5]

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords,tweets_stemmed
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i...",TechCabal talks with Bunmi Adeoye who heads in...,techcabal talks with bunmi adeoye who heads in...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1...",After 9 cities across 8 countries in Africa 15...,after 9 cities across 8 countries in africa 15...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...,RT TechCabal In this weeks edition of QuickFir...,rt techcabal in this weeks edition of quickfir...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...,Q1 22 is not over yet and many more startups s...,q1 22 is not over yet and many more startups s...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...,About 76 of the funding comes from the Big 4—N...,about 76 of the funding comes from the big 4—n...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...


In [74]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
tweets_df['tweets_lemmatized']=tweets_df['tweets_no_stopwords'].apply(lambda x:lemmatizer(x))
tweets_df[:5]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords,tweets_stemmed,tweets_lemmatized
0,Tue Feb 22 11:55:03 +0000 2022,"TechCabal talks with Bunmi Adeoye, who heads i...","TechCabal talks with Bunmi Adeoye, who heads i...",TechCabal talks with Bunmi Adeoye who heads in...,techcabal talks with bunmi adeoye who heads in...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...,[techcabal talks with bunmi adeoye who heads i...
1,Tue Feb 22 10:22:32 +0000 2022,"After 9 cities across 8 countries in Africa, 1...","After 9 cities across 8 countries in Africa, 1...",After 9 cities across 8 countries in Africa 15...,after 9 cities across 8 countries in africa 15...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...,[after 9 cities across 8 countries in africa 1...
2,Tue Feb 22 09:28:33 +0000 2022,RT @TechCabal: In this week's edition of #Quic...,RT @TechCabal: In this week's edition of #Quic...,RT TechCabal In this weeks edition of QuickFir...,rt techcabal in this weeks edition of quickfir...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...,[rt techcabal in this weeks edition of quickfi...
3,Tue Feb 22 09:11:52 +0000 2022,Q1 '22 is not over yet and many more startups ...,Q1 '22 is not over yet and many more startups ...,Q1 22 is not over yet and many more startups s...,q1 22 is not over yet and many more startups s...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...
4,Tue Feb 22 09:11:52 +0000 2022,About 76% of the funding comes from the Big 4—...,About 76% of the funding comes from the Big 4—...,About 76 of the funding comes from the Big 4—N...,about 76 of the funding comes from the big 4—n...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...,[about 76 of the funding comes from the big 4—...


In [75]:
from textblob import TextBlob
# function to calculate subjectivity
# def getSubjectivity(review):
#   return TextBlob(review).sentiment.subjectivity
# function to calculate polarity
def getPolarity(review):
  stem_text = [porter_stemmer.stem(word) for word in review]
  # return stem_text
  return TextBlob(stem_text[0]).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [76]:
sentiment_df = pd.DataFrame(tweets_df[['tweets', 'tweets_lemmatized']])
# fin_data['Subjectivity'] = fin_data['Lemma'].apply(getSubjectivity) 
sentiment_df['Polarity'] = sentiment_df['tweets_lemmatized'].apply(lambda x:getPolarity(x)) 
sentiment_df['Analysis'] = sentiment_df['Polarity'].apply(analysis)
sentiment_df

,tweets,tweets_lemmatized,Polarity,Analysis
0,"TechCabal talks with Bunmi Adeoye, who heads i...",[techcabal talks with bunmi adeoye who heads i...,0.000000,Neutral
1,"After 9 cities across 8 countries in Africa, 1...",[after 9 cities across 8 countries in africa 1...,0.000000,Neutral
2,RT @TechCabal: In this week's edition of #Quic...,[rt techcabal in this weeks edition of quickfi...,0.000000,Neutral
3,Q1 '22 is not over yet and many more startups ...,[q1 22 is not over yet and many more startups ...,0.500000,Positive
4,About 76% of the funding comes from the Big 4—...,[about 76 of the funding comes from the big 4—...,0.200000,Positive
...,...,...,...,...
195,"Join @arnergy's CEO, @femiadeyemo on #BFGU thi...",[join arnergys ceo femiadeyemo on bfgu this fr...,0.000000,Neutral
196,RT @genesystechhub: @TechCabal Canva designers...,[rt genesystechhub techcabal canva designers a...,0.000000,Neutral
197,Meta said that while it's winding down Express...,[meta said that while its winding down express...,-0.140278,Negative
198,"In April 2021, Spotify launched the Spotify EQ...",[in april 2021 spotify launched the spotify eq...,0.000000,Neutral


In [77]:
export_df = pd.DataFrame(sentiment_df[['tweets', 'Analysis']])
export_df[:7]

,tweets,Analysis
0,"TechCabal talks with Bunmi Adeoye, who heads i...",Neutral
1,"After 9 cities across 8 countries in Africa, 1...",Neutral
2,RT @TechCabal: In this week's edition of #Quic...,Neutral
3,Q1 '22 is not over yet and many more startups ...,Positive
4,About 76% of the funding comes from the Big 4—...,Positive
5,Just two months into 2022 and African startups...,Neutral
6,How much do you think African startups will ra...,Positive


In [78]:
export_df.to_csv("tweets.csv")